In [30]:
import emoji
import pandas as pd
import os
import re
import torch
import torch.nn as nn
import numpy as np
import sys
sys.path.append('../')

from torch.utils.data import DataLoader, Dataset
from transformers import BertModel, BertTokenizer
from sklearn.metrics import f1_score, accuracy_score
from tqdm.notebook import tqdm
from datasets import EvaluationDataset
from models import BertRegressionModelBase
from utils import decode_norm_lat, decode_norm_long, MAE_km

In [2]:
data = pd.read_csv('../data/geotexts_data_clean.csv')

C:\Users\kazza\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
def remove_url(text):
    return re.sub(r'http\S+', '', text)


In [4]:
def text_preprocess(text):
    text = emoji.demojize(text)
    text = remove_url(text)
    return text
    

In [5]:
#use multilingual bert
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
encoder = BertModel.from_pretrained("bert-base-multilingual-cased")
# encoder.load_state_dict(torch.load("bert_country_encoder2.pt"))

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
model = BertRegressionModelBase(encoder, tokenizer)
model.load_state_dict(torch.load("../data/geomodeldefault.pt"))

<All keys matched successfully>

In [47]:
def evaluate(texts, model, device="cuda", batch_size=32):
    res = []
    DEVICE = torch.device(device)
    texts = [text_preprocess(text) for text in texts]
    dataset = EvaluationDataset(texts)
    dataloader = DataLoader(dataset, batch_size = batch_size)
    for batch in tqdm(dataloader):
        with torch.no_grad():
            logits = model.forward(batch).cpu().numpy()
            for coords in logits:
                decoded_coords = (decode_norm_lat(coords[0]), decode_norm_long(coords[1]))
                res.append(decoded_coords)
    return res
        
    
    

In [49]:
preds = evaluate(data['text'][:10], model)

  0%|          | 0/1 [00:00<?, ?it/s]

In [44]:
labels = [(lat, long) for lat, long in zip(data['latitude'][:10], data['longitude'][:10])]

In [45]:
labels

[(-0.0332440668675476, -75.45230903823258),
 (-0.0332440668675476, -75.45230903823258),
 (-0.0332440668675476, -75.45230903823258),
 (-0.0332440668675476, -75.45230903823258),
 (-0.0332440668675476, -75.45230903823258),
 (-0.0332440668675476, -75.45230903823258),
 (-0.0332440668675476, -75.45230903823258),
 (-0.0332440668675476, -75.45230903823258),
 (-0.0332440668675476, -75.45230903823258),
 (-0.0332440668675476, -75.45230903823258)]

In [50]:
for predicted_coord, true_coords in zip(preds, labels):
    print(MAE_km(predicted_coord, true_coords))

333.5154782737243
297.4366233135621
280.10289530676243
362.9137139606802
534.6946273905927
704.4872239974543
422.9544866021564
170.11222841452278
384.69303409120505
391.7653611486112
